In [196]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from nltk.stem.snowball import RussianStemmer
import sklearn 
import string

from sklearn.feature_extraction.text import CountVectorizer

In [3]:
data = pd.DataFrame.from_csv('data/avito_train.tsv', sep='\t')

In [224]:
data.head(1)

,category,subcategory,title,description,attrs,price,is_proved,is_blocked,phones_cnt,emails_cnt,urls_cnt,close_hours
itemid,,,,,,,,,,,,
10000010,Транспорт,Автомобили с пробегом,"[toyot, ser]","[нов, оригинальн, линзова, оптик, ксенон, ближ...","[год, выпуск, тип, кузовакуп, пробег, коробк, ...",150000,NaN,0,0,0,0,0.03


In [225]:
train = data[:10000]
data.shape

(3995803, 12)

In [188]:
# просматриваем информацию в колонках
for column in train.columns:
    print("{: <20} {:} {: >10}".format(column, train[column].dtype, len(train[column].unique())))

category             object         10
subcategory          object         53
title                object       8542
description          object       9842
attrs                object       4405
price                int64       1344
is_proved            float64          3
is_blocked           int64          2
phones_cnt           int64          8
emails_cnt           int64          3
urls_cnt             int64          6
close_hours          float64       2124


In [226]:
# по таблице определяем категориальные string колонки
cat_cols = ['category','subcategory']

In [227]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# from sklearn.feature_extraction import DictVectorizer

enc = LabelEncoder()
train[cat_cols] = train[cat_cols].apply(lambda x: pd.factorize(x)[0])

In [228]:
# стемминг и знаки пунктуации
stemmer = RussianStemmer()
exclude = set(string.punctuation)
exclude = exclude | set(map(str,[1, 2, 3, 4, 5, 6, 7, 8, 9, 0]))

# разделение по словам без знаков пунктуации и цифр
def clear(title):
    temp = ''.join([i for i in str(title) if i not in exclude])
    temp = temp.split()
    temp = [stemmer.stem(i).lower() for i in temp]
    temp = [i for i in temp if len(i) > 2]
    return temp

In [229]:
train['title'] = train['title'].apply(clear)
train['description'] = train['description'].apply(clear)
train['attrs'] = train['attrs'].apply(clear)
train['td'] = train.title + train.description

In [231]:
train.is_blocked

itemid
10000010    0
10000025    0
10000094    0
10000101    0
10000132    0
10000152    0
10000160    0
10000224    0
10000243    0
10000247    0
10000255    0
10000263    0
10000287    0
10000290    0
10000299    0
10000309    0
10000316    0
10000317    1
10000338    0
10000348    0
10000369    0
10000372    0
10000390    0
10000397    0
10000420    0
10000422    0
10000423    0
10000440    0
10000445    0
10000459    0
           ..
10226733    0
10226777    0
10226824    0
10226838    0
10226863    0
10226865    0
10226921    0
10226927    0
10226928    0
10226976    0
10227002    0
10227007    0
10227031    1
10227040    0
10227053    0
10227083    0
10227099    0
10227106    0
10227113    0
10227150    1
10227153    0
10227155    0
10227223    0
10227246    0
10227253    0
10227279    1
10227294    0
10227299    1
10227300    0
10227361    0
Name: is_blocked, dtype: int64

In [207]:
words = list()
for elem in train.title:
    words.append(" ".join(elem))

In [216]:
len(words)

10000

In [214]:
# Initialize the "CountVectorizer" object, which is scikit-learn's
# bag of words tool.  
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = None,   \
                             max_features = None) 

train_data_features = vectorizer.fit_transform(words)

train_data_features = train_data_features.toarray()

In [217]:
np.unique(train_data_features[0])

array([0, 1], dtype=int64)

In [239]:
temp = train.is_blocked
temp = temp.reset_index(drop=True)

df_1 = pd.DataFrame(train_data_features)
df_1['target'] = temp

In [250]:
target = df_1.target

In [251]:
df_1 = df_1.drop('target', axis=1)

In [242]:
from sklearn.model_selection import train_test_split

In [253]:
X_train, X_test, y_train, y_test = train_test_split(df_1, target, test_size=0.3, random_state=5643)

In [255]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

In [256]:
clf = RandomForestClassifier(n_estimators=10)
scores = cross_val_score(clf, X_train, y_train, cv=5)
scores

array([ 0.95360457,  0.95428571,  0.95142857,  0.95714286,  0.94710508])

In [262]:
target.value_counts()[]

0    9322
1     678
Name: target, dtype: int64

=9322/10000